<!--NAVIGATION-->
< [Delineate streams](Streams.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Automate stream and catchment delineation](AutomateDelineation.ipynb) >

# 7. Délimitation des bassins versants
Dans cette section, nous allons délimiter le bassin versant d’un exutoire et tous les bassins versants dans le MNT.

Commençons par importer les bibliothèques et définir le chemin vers les données.

In [8]:
import os
from pcraster import *
os.chdir('./Data/')

## 7.1 Délimitation du bassin versant d'un exutoire

Nous devons d'abord identifier un exutoire sur le cours d'eau délimité à l'étape précédente. Vous pouvez utiliser le résultat de la méthode de l'ordre de Strahler ou de la méthode d'accumulation des flux pour trouver les pixels qui font partie de la rivière. Vous pouvez utiliser QGIS ou Aguila pour trouver les coordonnées. Dans Aguila, vous pouvez le faire en utilisant l'outil crosshair.

Vous pouvez utiliser les champs ci-dessous pour charger la carte de la rivière de la section précédente, visualiser avec Aguila et trouver les coordonnées d'un pixel sur la carte de la rivière que nous voulons utiliser comme exutoire pour le bassin versant que nous allons délimiter.

In [ ]:
river = readmap("stream8.map")

Visualiser la carte de la rivière

Utilisons 288880.648,5675880.258 comme coordonnées de l'exuctoire.
PCRaster est fourni avec un outil <code>col2map</code>, qui lit un fichier texte au format:

`x y id`

et le convertit en carte PCRaster. Vous pouvez lancer l'outil depuis la ligne de commande, mais ici nous allons l'intégrer dans une fonction Python. Vous trouverez plus d'informations sur l'outil `col2map` dans la [documentation de PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/app_col2map.html)

L'outil a des arguments pour le type de données et a besoin d'une carte clone. Dans notre cas, la sortie sera un raster nominal avec un pixel qui a un numéro d'identification. Pour le clone, nous pouvons utiliser la carte de la rivière.

In [ ]:
def col2map(x,y,id,datatype,clone):
    with open('location.txt', 'w') as f:
        f.write(str(x) + ' ' + str(y) + ' ' + str(id))
    cmd = 'col2map -{0} location.txt location.map --clone {1}'.format(datatype,clone)
    print(cmd)
    os.system(cmd)
    Map = readmap('location.map')
    return Map

x = 288880.648
y = 5675880.258
id = 1
datatype = 'N'
clone = 'stream8.map'
Outlet = col2map(x,y,id,datatype,clone)
aguila(Outlet,river)

La fonction créé un fichier texte. Nous avons écrit une chaîne de caractères avec "x y id" dans le fichier. Ensuite, la commande a été construite comme une chaîne de caractères (string) et nous avons exécuté la commande string avec <code>os.system</code>.

Nous avons également imprimé la commande string après avoir exécuté le script. Cette commande aurait pu être tapée à l'invite de commande.

Maintenant que nous avons défini l’exutoire de notre rivière, nous pouvons délimiter le bassin versant qui contribue à cet exutoire en utilisant l'opération <code>catchment</code>. L'opération <code>catchment</code>  nécessite la carte de direction du flux et la carte de l’exutoire comme entrées. Plus d'informations sur l'opération <code>catchment</code>  peuvent être trouvées dans la [documentation de PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_catchment.html).

In [ ]:
FlowDirection = readmap('flowdir.map')
RurCatchment = catchment(FlowDirection,Outlet)

Visualiser le résultat avec Aguila.

## 7.2 Délimitation de tous les bassins versants dans le MNT

Il s’est agi pour le moment du bassin versant d'un exutoire donné. Si nous voulons dériver automatiquement tous les bassins versants dans le raster, nous devons d'abord trouver les emplacement (pit), qui peuvent être considérées comme des exutoires. Cela peut être fait avec l'opération <code>pit</code>. Plus d'informations sur l'opération <code>pit</code>  peuvent être trouvées dans la [documentation de PCRaster](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_pit.html). Elle nécessite uniquement la carte de direction des flux comme entrée.

In [ ]:
outlets = pit(FlowDirection)

Visualiser le résultat avec Aguila.

*Ou sont situé les emplacements (pits)?*

*Combien il y en a t-il?*

Maintenant nous pouvons utiliser ces exutoires (emplacements) dans l'opération <code>catchment</code> pour dériver tous les bassins versants dans le raster.

In [ ]:
Catchments = catchment(FlowDirection, outlets)

Visualiser le résultat avec Aguila.

*Que voyez vous?*

*Pourquoi y a-t-il autant de bassin versant à la limite de la zone?*

Dans la section suivante, nous allons combiner tout le code en un seul script pour automatiser le processus autant que possible.

<!--NAVIGATION-->
< [Delineate streams](Streams.ipynb) | [Contents](ContentsCatchmentDelineation.ipynb) | [Automate stream and catchment delineation](AutomateDelineation.ipynb) >